#Importing necessary libraries

In [ ]:
import pandas as pd
import csv
import math
import copy
import collections
import itertools
from collections import Counter
from itertools import combinations
from itertools import chain
from math import log2
from matplotlib import pyplot as plt
from IPython.display import display



In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
FILE_DIR='//content/drive/MyDrive/Data Mining/Assignment 2'

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
#filename = open('/content/gdrive/MyDrive/Data Mining/Assignment-1-Data.csv', 'r')
#file = csv.DictReader(filename)

data = pd.read_csv("/content/gdrive/MyDrive/Data Mining/Assignment 2/Assignment-1-Data.csv")
df = data.copy(deep=True)


# converting column data to list
# treat	debt lopen lrer bndvol gdpgvol pivol spreadvol
treat=df['treat'].tolist()
debt=df['debt'].tolist()
lopen=df['lopen'].tolist()
lrer=df['lrer'].tolist()
bndvol=df['bndvol'].tolist()
gdpgvol= df['gdpgvol'].tolist()
pivol=df['pivol'].tolist()
spreadvol=df['spreadvol'].tolist()

# printing lists
print('treat:', treat)
print('debt:', debt)
print('lopen:', lopen)
print('lrer:', lrer)
print('bndvol:', bndvol)
print('gdpgvol:', gdpgvol)
print('pivol:', pivol)
print('spreadvol:', spreadvol)



treat: [1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

###Calculating Mean, Median and Standard Deviation

Defining function to calculate *mean*




In [ ]:
def calculate_mean(numbers):
  sum = 0
  for i in range(len(numbers)):
    sum = sum+numbers[i]
  mean = sum/len(numbers)
  return mean

Defining function to calculate median

In [ ]:
def calculate_median(numbers):
  #numbers.sort()
  temp = copy.deepcopy(numbers)
  temp.sort()
  count = len(temp)
  if count%2==0: ##if the sequence is even number
    index = int(count/2)
    median = (temp[index-1]+temp[index])/2
  else: ## if the sequence is an odd numnber
    index = count//2
    median = temp[index]
  return median

Defining function to calculate Standard deviation

In [ ]:
def calculate_SD(numbers):
  mean = calculate_mean(numbers)
  n = len(numbers)
  sum = 0
  for i in range(n):
    sum+=(numbers[i]-mean)**2
  variance = sum/n
  SD = math.sqrt(variance)
  return SD

Defining a function to identify redundant attributes using Pierce correlation coefficient.

In [ ]:
def determine_correlation(df, column1, column2):
  A=df[column1].tolist()
  B=df[column2].tolist()
  a_mean=calculate_mean(A)
  b_mean=calculate_mean(B)
  r=0
  ra2=0
  rb2=0
  for i in range(len(A)):
    r+=(A[i]-a_mean)*(B[i]-b_mean)
    ra2+=(A[i]-a_mean)**2
    rb2+=(B[i]-b_mean)**2
  ra = math.sqrt(ra2)
  rb = math.sqrt(rb2)
  rab = r/(ra*rb)
  return rab

Each empty cell shows the correlation between the two attributes that are used as the row and the column names. If the cells are black then the two attributes are strongly correlated.

In [ ]:
independent_attr = ['debt', 'lopen', 'lrer', 'bndvol', 'gdpgvol',  'pivol', 'spreadvol']
correlation_matrix = pd.DataFrame(index=independent_attr, columns=independent_attr)
r_values=[]

for i, attr_i in enumerate(independent_attr):
  for j, attr_j in enumerate(independent_attr):
    rab = determine_correlation(df, attr_i, attr_j)
    correlation_matrix.loc[attr_i, attr_j] = round(rab,2)
    r_values.append(round(rab,2))

print(correlation_matrix)
corr_mat= correlation_matrix.copy()
#print(corr_mat)

#print(r_values)
r=list(set(r_values))

threshold = input("Enter threshold value: ")

def color_matrix(value):
  color = 'black' if abs(round(value,2))>=float(threshold) else 'white'
  return f'background-color: {color}; color:{color}; border: 1px black solid !important'

print("The threshold value: ", threshold)
corr_mat.style.apply(lambda x: x.map(color_matrix))



           debt lopen  lrer bndvol gdpgvol pivol spreadvol
debt        1.0 -0.13  0.12   0.21    0.05  0.01      0.02
lopen     -0.13   1.0 -0.08  -0.01    0.23 -0.02     -0.08
lrer       0.12 -0.08   1.0  -0.26    0.12 -0.08     -0.14
bndvol     0.21 -0.01 -0.26    1.0    0.13  0.54      0.55
gdpgvol    0.05  0.23  0.12   0.13     1.0  0.28      0.28
pivol      0.01 -0.02 -0.08   0.54    0.28   1.0      0.49
spreadvol  0.02 -0.08 -0.14   0.55    0.28  0.49       1.0
Enter threshold value: .5
The threshold value:  .5


,debt,lopen,lrer,bndvol,gdpgvol,pivol,spreadvol
debt,1.000000,-0.130000,0.120000,0.210000,0.050000,0.010000,0.020000
lopen,-0.130000,1.000000,-0.080000,-0.010000,0.230000,-0.020000,-0.080000
lrer,0.120000,-0.080000,1.000000,-0.260000,0.120000,-0.080000,-0.140000
bndvol,0.210000,-0.010000,-0.260000,1.000000,0.130000,0.540000,0.550000
gdpgvol,0.050000,0.230000,0.120000,0.130000,1.000000,0.280000,0.280000
pivol,0.010000,-0.020000,-0.080000,0.540000,0.280000,1.000000,0.490000
spreadvol,0.020000,-0.080000,-0.140000,0.550000,0.280000,0.490000,1.000000


In [ ]:
def break_dataset_in_2(df):
  l=len(df)
  l_test=math.ceil(0.1*l)
  l_train=l-l_test
  print(l)
  print(l_train)
  print(l_test)
  df_train=df.head(l_train)
  df_test=df.iloc[-l_test:]
  return df_train, df_test

In [ ]:
econ_train, econ_test = break_dataset_in_2(df)
Econ_train=econ_train.reset_index(drop=True)
Econ_test=econ_test.reset_index(drop=True)
print(Econ_train)
print(Econ_test)

414
372
42
     treat  debt  lopen   lrer  bndvol  gdpgvol  pivol  spreadvol
0        1  26.4   78.4   99.4     0.4      1.2    0.2        1.1
1        1  26.4   78.4   99.4     0.4      1.2    0.2        1.1
2        1   6.2   85.5   96.2     0.3      1.1    0.4        1.0
3        1  23.5   87.6   96.4     0.5      0.5    0.7        0.6
4        1  25.3   87.0   97.1     0.5      1.0    0.4        0.1
..     ...   ...    ...    ...     ...      ...    ...        ...
367      0  38.5   73.1  152.8     0.6      2.2    1.3        0.1
368      0  43.3   79.4  142.1     0.4      3.9    3.7        0.8
369      0  41.2   75.9  149.9     0.4      8.1    3.9        2.1
370      0  79.1   84.7  117.8     0.5      4.3    3.7        1.5
371      0  99.0   90.4   95.2     0.7      4.5    4.0        0.7

[372 rows x 8 columns]
    treat   debt  lopen   lrer  bndvol  gdpgvol  pivol  spreadvol
0       0  105.7   97.1  100.0     0.6      3.0    0.7        0.6
1       0  113.0  105.2  101.4     0.3   

Calculation of the transition probabilities of going from start state to 0/1 state

In [ ]:
def initial_transition_probabilities(df):
  treat=df['treat'].tolist()
  count_1=treat.count(1)
  count_0=treat.count(0)
  total = len(treat)
  probability_1=count_1/total
  probability_0=count_0/total
  #print(count_1,count_0)
  print("P(1|start):",probability_1)
  print("P(0|start):",probability_0)
  return probability_1, probability_0

In [ ]:
probability_1, probability_0 = initial_transition_probabilities(df)

P(1|start): 0.5193236714975845
P(0|start): 0.4806763285024155


In [ ]:
print(treat)

[1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [ ]:
def transition_probabilities(df):
  treat=df['treat'].tolist()
  tcount_1=treat.count(1)
  tcount_0=treat.count(0)
  #print("total 1 in treat",tcount_1)
  #print("total 0 in treat",tcount_0)
  count_1_0=0
  count_1_1=0
  count_0_0=0
  count_0_1=0
  if treat[-1]==1:
    tcount_1=tcount_1-1
  else:
    tcount_0=tcount_0-1
  #print("total 1 in treat",tcount_1)
  #print("total 0 in treat",tcount_0)
  for i in range(len(treat)-1):
    if treat[i]==1 and treat[i+1]==0:
      count_1_0+=1
    elif treat[i]==1 and treat[i+1]==1:
      count_1_1+=1
    elif treat[i]==0 and treat[i+1]==0:
      count_0_0+=1
    else:
      count_0_1+=1
  p_1_0 =  count_1_0/tcount_1
  p_1_1 =  count_1_1/tcount_1
  p_0_0 =  count_0_0/tcount_0
  p_0_1 =  count_0_1/tcount_0
  print("P(0|1) = ", p_1_0)
  print("P(1|1) = ", p_1_1)
  print("P(0|0) = ", p_0_0)
  print("P(1|0) = ", p_0_1)
  return p_1_0, p_1_1, p_0_0, p_0_1


In [ ]:
p_1_0, p_1_1, p_0_0, p_0_1 = transition_probabilities(df)

P(0|1) =  0.037209302325581395
P(1|1) =  0.9627906976744186
P(0|0) =  0.9646464646464646
P(1|0) =  0.03535353535353535


Creating Set1 and Set0

In [ ]:
def oneDList(x):
    return list(itertools.chain(*x))

In [ ]:
def create_set_n_emission(df):
  treat=df['treat'].tolist()
  df_1 = df[df['treat']==1]
  df_0 = df[df['treat']==0]
  print(df_1)
  print(df_0)
  df_temp1=df_1.drop(columns='treat',axis=1)
  df_temp0=df_0.drop(columns='treat',axis=1)
  a1=df_temp1.values.tolist()
  set1=oneDList(a1)
  print(set1)
  emission1= set(set1)
  print(emission1)

  a0=df_temp0.values.tolist()
  set0=oneDList(a0)
  print(set0)
  emission0=set(set0)
  print(emission0)
  return set1, emission1, set0, emission0


In [ ]:
set1, emission1, set0, emission0 = create_set_n_emission(df)

     treat  debt  lopen  lrer  bndvol  gdpgvol  pivol  spreadvol
0        1  26.4   78.4  99.4     0.4      1.2    0.2        1.1
1        1  26.4   78.4  99.4     0.4      1.2    0.2        1.1
2        1   6.2   85.5  96.2     0.3      1.1    0.4        1.0
3        1  23.5   87.6  96.4     0.5      0.5    0.7        0.6
4        1  25.3   87.0  97.1     0.5      1.0    0.4        0.1
..     ...   ...    ...   ...     ...      ...    ...        ...
394      1  55.2   68.8  94.0     0.5      0.6    0.8        0.4
395      1  72.5   69.3  86.3     0.5      0.6    0.8        0.4
396      1  23.5   90.1  91.9     0.0      1.4    0.3        1.6
397      1  23.5   90.1  91.9     0.0      1.4    0.3        1.6
398      1  26.4   98.2  89.3     0.7      1.9    0.4        0.8

[215 rows x 8 columns]
     treat  debt  lopen   lrer  bndvol  gdpgvol  pivol  spreadvol
5        0  36.3   86.5  100.5     0.4      1.0    0.5        0.1
6        0  25.3   90.9  101.8     0.4      0.6    0.4        0.

In [ ]:
def caclculate_probablity(value,emission, set):
  probality_list = []
  #print("CR:",len(set))
  #print("Card(Emissions(Set))",len(emission))
  if value in emission:
    #print(f"There are {set.count(value)} of {value}")
    probablity_value= set.count(value)/len(set)

  else:
    probablity_value=0
  return probablity_value

In [ ]:
p1=caclculate_probablity(26.4,emission0,set0)
p1

0

In [ ]:
independent_attr = ['debt', 'lopen', 'lrer', 'bndvol', 'gdpgvol',  'pivol', 'spreadvol']

##b) Viterbi algorithm

In [ ]:
def prediction(o0,o1,z0,z1):
  max_value=max(o0,o1,z0,z1)
  if max_value==o0:
    prediction_value = 0
  elif max_value==o1:
    prediction_value = 1
  elif max_value==z1:
    prediction_value = 1
  else:
    prediction_value = 0
  return prediction_value

In [ ]:
p_list=[]

In [ ]:
def hmm(df):
 prediction_list=[]
 probability_1, probability_0 = initial_transition_probabilities(df)
 p_1_0, p_1_1, p_0_0, p_0_1 = transition_probabilities(df)
 set1, emission1, set0, emission0 = create_set_n_emission(df)
 df_hmm=df.drop(columns='treat',axis=1)
 for i in range(len(df_hmm)):
    for j in range(len(independent_attr)):
      if j==0:
        value=df_hmm.loc[i][j]
        p0=caclculate_probablity(value,emission0, set0)
        p1=caclculate_probablity(value,emission1, set1)
        o1=probability_1*p1
        z0=probability_0*p0
        #max_initial=max(o1,z0)
        if o1>z0:
          #print("start-->1")
          prediction_value = 1
        else:
          #print("start-->0")
          prediction_value = 0
      else:
        value=df_hmm.loc[i][j]

        p0=caclculate_probablity(value,emission0, set0)
        p1=caclculate_probablity(value,emission1, set1)
        o0=o1*p1*p_1_0
        o1=o1*p1*p_1_1
        z1=z0*p0*p_0_1
        z0=z0*p0*p_0_0
        prediction_value=prediction(o0,o1,z0,z1)
        #p_list.append(prediction_value)
        #if j==len(independent_attr):
          #prediction_value=prediction(o0,o1,z0,z1)
    prediction_list.append(prediction_value)
 print(prediction_list)
 return prediction_list

In [ ]:
predicted_treat=hmm(Econ_test)

P(1|start): 0.21428571428571427
P(0|start): 0.7857142857142857
P(0|1) =  0.1111111111111111
P(1|1) =  0.8888888888888888
P(0|0) =  0.96875
P(1|0) =  0.03125
    treat  debt  lopen   lrer  bndvol  gdpgvol  pivol  spreadvol
18      1  35.1   68.4  100.0     0.6      1.1    0.8        0.7
19      1  20.3   69.8  100.6     0.5      1.0    0.7        0.8
20      1  20.8   68.3  100.3     0.2      0.9    0.8        1.3
21      1  20.9   67.7   98.9     0.5      0.5    0.1        0.8
22      1  55.2   68.8   94.0     0.5      0.6    0.8        0.4
23      1  72.5   69.3   86.3     0.5      0.6    0.8        0.4
24      1  23.5   90.1   91.9     0.0      1.4    0.3        1.6
25      1  23.5   90.1   91.9     0.0      1.4    0.3        1.6
26      1  26.4   98.2   89.3     0.7      1.9    0.4        0.8
    treat   debt  lopen   lrer  bndvol  gdpgvol  pivol  spreadvol
0       0  105.7   97.1  100.0     0.6      3.0    0.7        0.6
1       0  113.0  105.2  101.4     0.3      1.7    0.7       

In [ ]:
def hmm(df):
 prediction_list=[]
 probability_1, probability_0 = initial_transition_probabilities(df)
 p_1_0, p_1_1, p_0_0, p_0_1 = transition_probabilities(df)
 set1, emission1, set0, emission0 = create_set_n_emission(df)
 df_hmm=df.drop(columns='treat',axis=1)
 for i in range(len(df_hmm)):
    for j in range(len(independent_attr)):
      if j==0:
        value=df_hmm.loc[i][j]
        p0=caclculate_probablity(value,emission0, set0)
        p1=caclculate_probablity(value,emission1, set1)
        o1=probability_1*p1
        z0=probability_0*p0
        #max_initial=max(o1,z0)
        if o1>z0:
          #print("start-->1")
          prediction_value = 1
        else:
          #print("start-->0")
          prediction_value = 0
      else:
        value=df_hmm.loc[i][j]

        p0=caclculate_probablity(value,emission0, set0)
        p1=caclculate_probablity(value,emission1, set1)
        o0=o1*p1*p_1_0
        o1=o1*p1*p_1_1
        z1=z0*p0*p_0_1
        z0=z0*p0*p_0_0
        prediction_value=prediction(o0,o1,z0,z1)
        #p_list.append(prediction_value)
        #if j==len(independent_attr):
          #prediction_value=prediction(o0,o1,z0,z1)
    prediction_list.append(prediction_value)
 print(prediction_list)
 return prediction_list

##d) Count how many correct and incorrect predictions

In [ ]:
print(predicted_treat)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [ ]:
len(predicted_treat)

42

In [ ]:
actual_treat=Econ_test['treat'].tolist()
len(actual_treat)

42

In [ ]:
print(actual_treat)
print(predicted_treat)
x=0
y=0
z=0
w=0
for i in range(len(actual_treat)):
  if actual_treat[i] == 1 and predicted_treat[i]==1:
    x += 1
  if actual_treat[i] == 1 and predicted_treat[i]==0:
    y += 1
  if actual_treat[i] == 0 and predicted_treat[i]==1:
    z +=1
  if actual_treat[i] == 0 and predicted_treat[i]==0:
    w +=1
print("x = ",x)
print("y = ",y)
print("z = ",z)
print("w = ",w)
correct_prediction = x+w
print("correct_prediction = ", correct_prediction)

accuracy = (100*(x+w))/len(Econ_test)
print(f"{round(float(accuracy),2)}% is the percent of the correct prediction")

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
x =  9
y =  0
z =  0
w =  33
correct_prediction =  42
100% is the percent of the correct prediction
